In [1]:

import pandas as pd
import numpy as np

data = pd.read_csv("./cl2.csv")

In [2]:
#assigning values to features as X and target as y
X=data.drop(["fetal_health"],axis=1)
y=data["fetal_health"]

In [3]:
y.value_counts()

0    921
2     94
1     48
Name: fetal_health, dtype: int64

In [4]:
from sklearn.model_selection import train_test_split

# splitting data
X_train, X_test, y_train, y_test = train_test_split(
                X,y,
                test_size=0.2,
                random_state=42)

print("Shape of training set:", X_train.shape)
print("Shape of test set:", X_test.shape)

Shape of training set: (850, 22)
Shape of test set: (213, 22)


In [34]:
from xgboost import XGBClassifier 

xgb = XGBClassifier()

xgb.fit(X_train, y_train)
print()
y_pred = xgb.predict(X_test)

In [35]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn import metrics 
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test,y_pred))

Confusion Matrix: 
 [[176   3   0]
 [  3   7   0]
 [  1   0  23]]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       179
           1       0.70      0.70      0.70        10
           2       1.00      0.96      0.98        24

    accuracy                           0.97       213
   macro avg       0.89      0.88      0.89       213
weighted avg       0.97      0.97      0.97       213



In [43]:
xgb_train = xgb.DMatrix(X_train, y_train, enable_categorical=True)
xgb_test = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [42]:
import warnings
from logging import INFO
import xgboost as xgb

import flwr as fl
from flwr_datasets import FederatedDataset
from flwr.common.logger import log
from flwr.common import (
    Code,
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    GetParametersIns,
    GetParametersRes,
    Parameters,
    Status,
)
from flwr_datasets.partitioner import IidPartitioner

from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics 
warnings.filterwarnings("ignore", category=UserWarning)

# def transform_dataset_to_dmatrix(data: Union[Dataset, DatasetDict]) -> xgb.core.DMatrix:
#     """Transform dataset to DMatrix format for xgboost."""
#     x = data["inputs"]
#     y = data["label"]
#     new_data = xgb.DMatrix(x, label=y)
#     return new_data

# # Train/test splitting
# train_data, valid_data, num_train, num_val = X_train, X_test, y_train, y_test
num_train = 850
num_val = 213


xgb_train = xgb.DMatrix(X_train, y_train, enable_categorical=True)
xgb_test = xgb.DMatrix(X_test, y_test, enable_categorical=True)

# Reformat data to DMatrix for xgboost
log(INFO, "Reformatting data...")
# train_dmatrix = transform_dataset_to_dmatrix(train_data)
# valid_dmatrix = transform_dataset_to_dmatrix(valid_data)
train_dmatrix = xgb_train
valid_dmatrix = xgb_test


# Hyper-parameters for xgboost training
num_local_round = 1
params = {
    "objective": "multi:softprob",
    "eta": 0.5,  # Learning rate
    "max_depth": 100,
    "eval_metric": "mlogloss",
    # "nthread": 16,
    # "num_parallel_tree": 1,
    # "subsample": 1,
    # "tree_method": "hist",
    'num_class':3
}


# Define Flower client
class XgbClient(fl.client.Client):
    def __init__(self):
        self.bst = None
        self.config = None

    def get_parameters(self, ins: GetParametersIns) -> GetParametersRes:
        _ = (self, ins)
        return GetParametersRes(
            status=Status(
                code=Code.OK,
                message="OK",
            ),
            parameters=Parameters(tensor_type="", tensors=[]),
        )

    def _local_boost(self):
        # Update trees based on local training data.
        for i in range(num_local_round):
            self.bst.update(train_dmatrix, self.bst.num_boosted_rounds())

        # Extract the last N=num_local_round trees for sever aggregation
        bst = self.bst[
            self.bst.num_boosted_rounds()
            - num_local_round : self.bst.num_boosted_rounds()
        ]

        return bst

    def fit(self, ins: FitIns) -> FitRes:
        if not self.bst:
            # First round local training
            log(INFO, "Start training at round 1")
            bst = xgb.train(
                params,
                train_dmatrix,
                num_boost_round=num_local_round,
                evals=[(valid_dmatrix, "validate"), (train_dmatrix, "train")],
            )
            self.config = bst.save_config()
            self.bst = bst
        else:
            for item in ins.parameters.tensors:
                global_model = bytearray(item)

            # Load global model into booster
            self.bst.load_model(global_model)
            self.bst.load_config(self.config)

            bst = self._local_boost()

        local_model = bst.save_raw("json")
        local_model_bytes = bytes(local_model)

        return FitRes(
            status=Status(
                code=Code.OK,
                message="OK",
            ),
            parameters=Parameters(tensor_type="", tensors=[local_model_bytes]),
            num_examples=num_train,
            metrics={},
        )

    def evaluate(self, ins: EvaluateIns) -> EvaluateRes:
        eval_results = self.bst.eval_set(
            evals=[(valid_dmatrix, "valid")],
            iteration=self.bst.num_boosted_rounds() - 1,
        )
        auc = round(float(eval_results.split("\t")[1].split(":")[1]), 4)
        
        preds = self.bst.predict(valid_dmatrix)
        y_pred = np.multiply(preds,100)
        y_pred = y_pred.astype(int)
        print(y_pred, "nigga")
        # a = [0 if i == 0 else 1 if i == 100 else 2 for i in y_pred]
        # print(classification_report(y_test, a))
        # print(metrics.classification_report(y_test,a))
        


        return EvaluateRes(
            status=Status(
                code=Code.OK,
                message="OK",
            ),
            loss=0.0,
            num_examples=num_val,
            metrics={"error": auc},
        )


# Start Flower client
fl.client.start_client(server_address="127.0.0.1:8080", client=XgbClient().to_client())

INFO flwr 2024-05-12 14:03:29,373 | 2106825142.py:42 | Reformatting data...
INFO flwr 2024-05-12 14:03:29,375 | grpc.py:52 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2024-05-12 14:03:29,389 | connection.py:55 | ChannelConnectivity.IDLE
DEBUG flwr 2024-05-12 14:03:29,392 | connection.py:55 | ChannelConnectivity.READY
INFO flwr 2024-05-12 14:03:29,394 | 2106825142.py:96 | Start training at round 1


[0]	validate-mlogloss:0.56801	train-mlogloss:0.54077
[[60 20 19]
 [28 50 20]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [20 20 59]
 [60 20 19]
 [37 25 37]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [20 20 59]
 [60 20 19]
 [59 20 19]
 [60 20 19]
 [59 20 19]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [20 20 59]
 [22 56 20]
 [60 20 19]
 [60 20 19]
 [59 20 19]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [20 20 59]
 [60 20 19]
 [60 20 19]
 [20 20 59]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [20 20 59]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [54 21 23]
 [60 20 19]
 [60 20 19]
 [46 37 15]
 [59 20 19]
 [60 20 19]
 [60 20 19]
 [42 41 16]
 [60 20 19]
 [60 20 19]
 [26 22 50]
 [60 20 19]
 [22 56 20]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [22 19 58]
 [55 26 18]
 [60 20 19]
 [60 20 19]
 [60 20 19]
 [42 41 16]

DEBUG flwr 2024-05-12 14:03:30,343 | connection.py:220 | gRPC channel closed
INFO flwr 2024-05-12 14:03:30,345 | app.py:398 | Disconnect and shut down


[[99  0  0]
 [33 49 16]
 [99  0  0]
 [89  7  2]
 [92  4  2]
 [99  0  0]
 [88  4  6]
 [99  0  0]
 [99  0  0]
 [89  7  2]
 [99  0  0]
 [89  7  2]
 [99  0  0]
 [99  0  0]
 [99  0  0]
 [25 12 62]
 [89  7  2]
 [ 8 52 38]
 [99  0  0]
 [99  0  0]
 [99  0  0]
 [99  0  0]
 [99  0  0]
 [99  0  0]
 [ 0 22 77]
 [99  0  0]
 [99  0  0]
 [99  0  0]
 [99  0  0]
 [99  0  0]
 [99  0  0]
 [99  0  0]
 [ 0 22 77]
 [ 2 84 13]
 [99  0  0]
 [99  0  0]
 [89  2  7]
 [99  0  0]
 [99  0  0]
 [99  0  0]
 [99  0  0]
 [99  0  0]
 [75 14  9]
 [ 5 16 77]
 [99  0  0]
 [99  0  0]
 [ 0 22 77]
 [90  3  5]
 [99  0  0]
 [99  0  0]
 [99  0  0]
 [ 0 22 77]
 [99  0  0]
 [99  0  0]
 [99  0  0]
 [ 0 22 77]
 [99  0  0]
 [99  0  0]
 [88  7  3]
 [89  7  2]
 [99  0  0]
 [99  0  0]
 [72 23  3]
 [99  0  0]
 [99  0  0]
 [ 4 19 76]
 [60 26 12]
 [21 53 25]
 [99  0  0]
 [99  0  0]
 [99  0  0]
 [86  9  4]
 [99  0  0]
 [ 0 18 81]
 [98  1  0]
 [99  0  0]
 [97  1  0]
 [99  0  0]
 [72 23  3]
 [83 10  6]
 [ 0 85 13]
 [79 15  5]
 [11 53 34]
 [98